# Preparing count matrices and variable genes for cNMF

In [2]:
library(KernSmooth)
library(reshape2)
library(Rtsne)

library(ggplot2)
library(RColorBrewer)
library(gplots)
library(ggrepel)
library(dplyr)
library(plyr)
library(DescTools)

library(Seurat)
library(matrixStats)
library(tidyr)

library(dbscan)


source("../sc_analysis_palettes_and_helpers.r")

#wd <- '/local/users/nfresma/'

wd <- './nmf_data/'



## Write count matrixes and variable genes lists for various sets of NB cell datasets:

In [ ]:
# Load NB cell Seurat object

dat <- readRDS('seur_all_tumors_NB_cells.rds')

### For entire dataset

In [5]:
# Calculate variable genes and remove variable genes only found in certain datasets or specific to contaminating blood cells

# Find genes that are differentially expressed between datasets to a certain cut-off
Idents(dat) <- "dataset"

dat <- FindVariableFeatures(object = dat,selection.method = 'vst', nfeatures = 2000)

to.regress <- FindAllMarkers(object = dat, only.pos = TRUE, min.pct = 0.8, logfc.threshold = 0.3)
to.regress <- to.regress[to.regress$cluster %like any% 'multi_%',]
to.regress <- to.regress[to.regress$pct.1 > 0.9,]
to.regress <- to.regress$gene

# Load genes associated with the cell cycle that should also be excluded for and NMF.

to.regress.cellcycle <- read.csv("../transcriptome_analysis/data/zfish_prolif_markers.csv")
to.regress.cellcycle <- as.character(to.regress.cellcycle$x)
to.regress.cellcycle

# Get genes specific to blood cell types that are contaminants in NB cell data to remove those from variable genes before clustering and NMF.
regress_blood <- c("hbba1","hbba1.1","hbae1","hbbe2","hbaa1","hbaa2","hbba2","hbbe1.1",
                   "lyz","lect2l","mmp9","npsn","srgn",
                   "cd74a","cd74b","dusp2",
                   "c1qb","c1qc","grn1","marco",
                   "tox","zap70","sla2","igic1s1",
                   "nkl.1","nkl.4")


var_feats <- VariableFeatures(object = dat)

var_feats <- var_feats[!var_feats %in% c("mCherry", "EGFP","MYCN", "LMO1","dTomato")] # Remove transgenes as they're not all present in all tumour samples.
var_feats <- var_feats[!var_feats %in% to.regress]
var_feats <- var_feats[!var_feats %in% to.regress.cellcycle]
var_feats <- var_feats[!var_feats %in% regress_blood]


Calculating cluster mycn_tum_01

Calculating cluster mycn_tum_02

Calculating cluster mycnlmo1_ventral_tum

Calculating cluster mycnlmo1_kidney_tum

Calculating cluster healthyhk_ctrl

Calculating cluster lmo1_ctrl

Calculating cluster mycn_tum_04

Calculating cluster multi_seq_1_dr_tum

Calculating cluster multi_seq_2

Calculating cluster multi_seq_3

Calculating cluster multi_4_allo1

Calculating cluster multi_allos_1_tum

Calculating cluster multi_bAllos_1_tum

Calculating cluster multi_bAllos_2_tum

Calculating cluster multi_bAllos_3_tum

Calculating cluster multi_bAllos_4_tum

Calculating cluster multiseq_17



[1] "atad2"    "blm"      "brip1"    "casp8ap2" "cdc45"    "cdc6"    
  [7] "cdca7a"   "chaf1b"   "dscc1"    "dtl"      "e2f8"     "exo1"    
 [13] "fen1"     "gins2"    "gmnn"     "hells"    "mcm2"     "mcm4"    
 [19] "mcm5"     "mcm6"     "msh2"     "nasp"     "pcna"     "pola1"   
 [25] "pold3"    "prim1"    "rad51"    "rad51ap1" "rfc2"     "rpa2"    
 [31] "rrm1"     "rrm2"     "slbp"     "tipin"    "tyms"     "ubr7"    
 [37] "uhrf1"    "unga"     "ungb"     "usp1"     "wdr76"    "anln"    
 [43] "anp32e"   "aurka"    "birc5a"   "birc5b"   "bub1"     "cbx5"    
 [49] "ccnb2"    "cdc20"    "cdca8"    "cdk1"     "cenpe"    "cenpf"   
 [55] "ckap2l"   "ckap5"    "cks1b"    "cks2"     "ctcf"     "dlgap5"  
 [61] "ect2"     "g2e3"     "gas2l3"   "gtse1"    "hmgb2a"   "hmgb2b"  
 [67] "hmmr"     "kif11"    "kif20ba"  "kif20bb"  "kif23"    "kif2c"   
 [73] "lbr"      "mki67"    "ncapd2"   "ndc80"    "nek2"     "nuf2"    
 [79] "nusap1"   "rangap1a" "rangap1b" "tacc3"    "tmpoa"    "tmpob"   
 [85] "top2a"    "tpx2"     "ttk"      "tubb4b"   "ube2c"    "stmn1a"  
 [91] "mki67"    "plk1"     "top2a"    "nusap1"   "ccnb1"    "ube2c"   
 [97] "aspm"     "tpx2"     "kifc1"    "smc2"     "hmgb2a"   "hmgn2"   
[103] "DUT"      "tubb2b"   "tuba8l4"  "stmn1b"   "pcna"     "hmgb2b"

Sets of genes excluded:

In [12]:
print(unique(to.regress))
print(to.regress.cellcycle)

 [1] "eef1b2"            "rpl10"             "rplp0"            
 [4] "tpt1"              "hsp90ab1"          "rps6"             
 [7] "rpl19"             "rack1"             "rpl17"            
[10] "rps5"              "rpl9"              "rps8a"            
[13] "rpl7"              "EGFP"              "rpl3"             
[16] "rpl10a"            "FO704736.1"        "mibp2"            
[19] "CR318588.1"        "CR318588.4"        "mt-atp6"          
[22] "eef1db"            "actb2"             "rpl22l1"          
[25] "rpl12"             "rps27.2"           "h2afx1"           
[28] "nme2b.1"           "eef1g"             "eef2b"            
[31] "rpl18a"            "rpl6"              "ppiaa"            
[34] "rpl4"              "cfl1"              "b2m"              
[37] "rplp1"             "si:ch211-5k11.8"   "NC-002333.17"     
[40] "NC-002333.4"       "hbba1.1"           "rgs5a"            
[43] "zgc:158463"        "CT027638.1"        "LMO1"             
[46] "mCherry"           

In [ ]:
# Get count matrix and remove cells that have zero counts of highly variable genes.
# These will otherwise produce an error when running cNMF

allcells_eset <- t(as.matrix(GetAssayData(dat, slot = 'counts')))

cell_counts <- rowSums(allcells_eset[,colnames(allcells_eset) %in% var_feats])
names(cell_counts) <- rownames(allcells_eset)

cell_counts <- names(cell_counts[cell_counts == 0])
allcells_eset <- allcells_eset[!rownames(allcells_eset) %in% cell_counts,]


In [ ]:
write.table(var_feats, paste0(wd, "./nmf_data/allmerged_nrblstm_varGenes.txt"), quote = F, sep = "\t", row.names = F, col.names = F)
write.table(allcells_eset, paste0(wd, "./nmf_data/allmerged_nrblstm_counts.tsv", quote = F, sep = "\t")

## For MULTI-seq datasets

In [13]:
# Load genes associated with the cell cycle that should also be excluded for clustering
to.regress.cellcycle <- read.csv("../transcriptome_analysis/data/zfish_prolif_markers.csv")
to.regress.cellcycle <- as.character(to.regress.cellcycle$x)

# Get genes specific to blood cell types that are contaminants in NB cell data
regress_blood <- c("hbba1","hbba1.1","hbae1","hbbe2","hbaa1","hbaa2","hbba2","hbbe1.1",
                   "lyz","lect2l","mmp9","npsn","srgn",
                   "cd74a","cd74b","dusp2",
                   "c1qb","c1qc","grn1","marco",
                   "tox","zap70","sla2","igic1s1",
                   "nkl.1","nkl.4")


In [ ]:
# Per dataset:
# Subset full dataset -> Merge sets of cells derived from the same tumour samples but processed on separate 10x lanes.
# Find variable genes.
# Remove known cell cycle markers, contaminating blood / immune cell genes and transgenes from variable genes list.
# Write out count matrix and variable genes.

# MULTI-seq datasets to loop through
datasets <- c('multi_seq_2_s1','multi_seq_3_s1','multi_4_allo1_1','multi_allos_1_tum',
              'multi_bAllos_1_tum','multi_bAllos_2_tum','multi_bAllos_3_tum','multi_bAllos_4_tum','multiseq_17')

Idents(dat) <- "orig.ident"

for(i in 1:length(datasets)){
    
    subname <- datasets[i]
    
    if(subname == "multi_seq_2_s1"){
        subdats <- c("multi_seq_2_s1","multi_seq_2_s2")
        subname <- substr(subname, 1,(nchar(subname)-3))
    }else if(subname == "multi_seq_3_s1"){
        subdats <- c("multi_seq_3_s1","multi_seq_3_s2")
        subname <- substr(subname, 1,(nchar(subname)-3))
    }else if(subname == "multi_4_allo1_1"){
        subdats <- c("multi_4_allo1_1","multi_4_allo1_2")
        subname <- substr(subname, 1,(nchar(subname)-2))
    }else if(subname == "multi_bAllos_4_tum"){
        subdats <- c("multi_bAllos_4_PTs_S1","multi_bAllos_4_PTs_S2")
    }else if(subname == "multiseq_17"){
        subdats <- c("multiseq_17_S1","multiseq_17_S2")
    }else{
        subdats <- datasets[i]
    }
    
    sub <- subset(dat, idents = subdats)
        
    allcells_eset <- t(as.matrix(GetAssayData(sub, slot = 'counts')))
    
    sub <- NormalizeData(sub)
    all.genes <- rownames(x = sub)
    sub <- ScaleData(object = sub, features = all.genes)
    sub <- FindVariableFeatures(object = sub,selection.method = 'vst', nfeatures = 2000)
    var_feats <- VariableFeatures(object = sub)
    var_feats <- var_feats[!var_feats %in% c("mCherry", "EGFP","MYCN", "LMO1","dTomato")]
    var_feats <- var_feats[!var_feats %in% to.regress.cellcycle]
    var_feats <- var_feats[!var_feats %in% regress_blood]

    
    write.table(allcells_eset, paste0(wd,"./nmf_data/",subname,"_nrblstm_counts.tsv"), quote = F, sep = "\t")
    write.table(var_feats, paste0(wd,"./nmf_data/",subname,"_nrblstm_varGenes.txt"), quote = F, sep = "\t", row.names = F, col.names = F)
    
    rm(allcells_eset)
    rm(sub)
    rm(subname)
}

## For individual tumor datasets

In [7]:
# Load genes associated with the cell cycle that should also be excluded for clustering
to.regress.cellcycle <- read.csv("/data/junker/users/nfresma/zfish_prolif_markers.csv")
to.regress.cellcycle <- as.character(to.regress.cellcycle$x)

# Get genes specific to blood cell types that are contaminants in NB cell data
regress_blood <- c("hbba1","hbba1.1","hbae1","hbbe2","hbaa1","hbaa2","hbba2","hbbe1.1",
                   "lyz","lect2l","mmp9","npsn","srgn",
                   "cd74a","cd74b","dusp2",
                   "c1qb","c1qc","grn1","marco",
                   "tox","zap70","sla2","igic1s1",
                   "nkl.1","nkl.4")

In [29]:
# Per dataset:
# Subset full dataset. Only keep processing data, if dataset contains at least 100 NB cells.
# Find variable genes.
# Remove known cell cycle markers, contaminating blood / immune cell genes and transgenes from variable genes list.
# Write out count matrix and variable genes.

Idents(dat) <- "sample_all"

for(i in 1:length(unique(dat$sample_all))){
    
    
    subname <- unique(dat$sample_all)[i]
    sub <- subset(dat, idents = subname)
    
    
    if(ncol(sub) < 100){
        print(paste0('Dataset ',subname, ' is too small and will not be written to file for cNMF.'))
        next
    }
    
    allcells_eset <- t(as.matrix(GetAssayData(sub, slot = 'counts')))
    
    sub <- NormalizeData(sub, verbose = F)
    all.genes <- rownames(x = sub)
    sub <- FindVariableFeatures(object = sub,selection.method = 'vst', nfeatures = 2000, verbose = F)
    
    var_feats <- VariableFeatures(object = sub)
    var_feats <- var_feats[!var_feats %in% c("mCherry", "EGFP","MYCN", "LMO1","dTomato")]
    var_feats <- var_feats[!var_feats %in% to.regress.cellcycle]
    var_feats <- var_feats[!var_feats %in% to.regress]
    var_feats <- var_feats[!var_feats %in% regress_blood]

    
    write.table(allcells_eset, paste0(wd, "./nmf_data/", subname, "_nrblstm_counts.tsv"), quote = F, sep = "\t")
    write.table(var_feats, paste0(wd, "./nmf_data/", subname, "_nrblstm_varGenes.txt"), quote = F, sep = "\t", row.names = F, col.names = F)
    
    rm(allcells_eset)
    rm(sub)
    rm(subname)
}


[1] "Dataset EGFP_ctrl_1 is too small and will not be written to file for cNMF."
[1] "Dataset LMO1_ctrl_1 is too small and will not be written to file for cNMF."
[1] "Dataset LMO1_ctrl_m2 is too small and will not be written to file for cNMF."
[1] "Dataset MYCN_LMO1_vent_m7_1_1 is too small and will not be written to file for cNMF."
[1] "Dataset MYCN_LMO1_lat_m7_3_1 is too small and will not be written to file for cNMF."
